## Script for creating training sample
### Johan Spanners images

1) you need to flatten Johans file structure  
2) Do it on his drive in a new folder called; flatten_folder  
3) Rename each image acording to a new ID; JS + N_i  (JS = Johan Spanner)
4) Draw 5000 random images  
5) Save in new folder: sample_5000  
5) crate csv frame - that is csv with column names and row index  
6) column names = features  
7) row index = image ID  
8) codebook  
9) Page in code book showing the order of the column names so they don't have to scroll 

to mount external harddriv:
> sudo mkdir /mnt/f # f is just random letter for the mounting point  
> sudo mount -t drvfs d: /mnt/f # f is the letter choosen above, d is the drive name

Now you can access his files at location: **/mnt/f/'October Johan Archive'/Spanner**

You then have the folders Published, Raw, Submitted.  
Note to spanner: Something wrong in Pub 2006... Seems like raw or Pub

You know the file extension form the script CheckFileExytensions.ipynb


# Mounting the external harddrive

In [1]:
pw_path = '/mnt/c/Users/xpn381/Documents/data/lpw.txt'
file = open(pw_path, mode='r')
pw = file.read()

In [2]:
# uses pw as a var in terminall. Don't worry if it is already mounted
!echo $pw|sudo -S sudo -S mount -t drvfs d: /mnt/f
    
# !echo {password}|sudo -S {command}
# password can be a var here: https://stackoverflow.com/questions/44996933/using-sudo-inside-jupyter-notebooks-cell

[sudo] password for xpn381: mount: /mnt/f: d: already mounted on /mnt/f.


# Script

In [3]:
import os
import cv2
from IPython.utils import io # to capture outputs and errors
from iptcinfo3 import IPTCInfo
from PIL import Image
from PIL.TiffTags import TAGS # til raw
import rawpy # Måske bedre til raw 
import imageio # for saving NEF as jpgs
import re
import zipfile
import shutil
import numpy as np
import time
import matplotlib.pyplot as plt
import pickle

In [4]:
pkl_file = open('IPTC_keys.pkl', 'rb')

IPTC_keys = pickle.load(pkl_file)

pkl_file.close()

In [5]:
# external_dir =  "/mnt/f/OctoberJohanArchive"
external_dir =  "/mnt/f/OctoberJohanArchiveTest" # only the published images
flatten_dir =  "/mnt/f/OctoberJohanArchiveFlatten"

In [6]:
def image_check(img_path):
    
    """Only for testing! open and plots an image"""

    img_array = cv2.imread(img_path)
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

    plt.imshow(img_array)

    plt.show()

In [7]:
def count_files(external_dir):
    
    """Counts files in folder tree. Sanity check, and fits in the function flatten_dirs()"""
    
    count = 0
    
    for root, dirs, files in os.walk(external_dir):
        for name in files:
            count += 1
        
    return(count)

In [8]:
def mk_flatten_dir(flatten_dir):
    
    """Creates a new folder which is to be the location of the flattened folder tree. fits in the function flatten_dirs()"""
    
    if not os.path.exists(flatten_dir):
        os.mkdir(flatten_dir) # makes new dir                     
        
    # Or REPLACE old if exist
    else:
        shutil.rmtree(flatten_dir) # delets dir and content.. FOREVER!
        os.mkdir(flatten_dir) # makes new dir


In [9]:
def get_dt(meta_string):
    
    """Gets date and time from image meta data string. fits in the functions handl_..."""
                
    r = '[\d]{4}:[\d]{2}:[\d]{2} [\d]{2}:[\d]{2}:[\d]{2}'# time and, date. but is this robust? we will find out.
    
    date, time = re.search(r, meta_string).group().replace(':','').split(' ')
        
    return(date,time)

In [10]:
def handl_nef_n_cr2(old_path, new_path): # need to add new path in flatten folder eventually

    """Gets time and dats from NEF and CR2 formats. Then converts (and potientially moves) the files to .jpg. fits in function MoveConvertGetMeta"""
    
    # Rawpy works for CR2 and NEF
    raw = rawpy.imread(old_path)
    meta_stringN = str(raw.extract_thumb())

    date, time = get_dt(meta_stringN)
#     print(date, time)
    # image_check(test_imgRawNEF) 

    # Special treatment for NEF (might also work for others)
#     new_path = old_path.split('.')[0]+'TEST.jpg'  # Slick, but need to add new path in flatten folder eventually
    rgb = raw.postprocess()
    imageio.imsave(new_path, rgb)
    
    raw.close()
    
#     image_check(new_path)
    return(date, time)

In [11]:
def handl_tif(old_path, new_path):  # need to add new path in flatten folder eventually
    
    """Gets time and dats from tif format. Then converts (and potientially moves) the files to .jpg. fits in function MoveConvertGetMeta"""

    
    # PIL works for tiff
    img = Image.open(old_path)
    meta_stringT = str(img.tag.tagdata) # seems, there are slightly different timestamp. 2 hours dif it is a timezone thingy - you get the right one

    date, time = get_dt(meta_stringT)
#     print(date, time)
#     print(img.size)

#     new_path = old_path.split('.')[0]+'TEST.jpg'

    img.save(new_path) # turns it into an jpg, but does not create exif
    
    img.close()
    
#     image_check(new_path)
    return(date, time)

In [12]:
def handl_jpg(old_path, new_path): # need to add new path in flatten folder eventually

    """Gets time and dats from jpg format. Then converts(sic) (and potientially moves) the files to .jpg. fits in function MoveConvertGetMeta"""
    
    img = Image.open(old_path)
    meta_stringJ = str(img.info['exif'])

    date, time = get_dt(meta_stringJ)
#     print(date, time)
#     print(img.size)

#     new_path = old_path.split('.')[0]+'TEST.jpg'

    img.save(new_path, exif = img.info['exif']) # turns it into an jpg, and maintain exif
    img.close()

#     image_check(new_path)
    return(date, time)

get_IPCT_data seems to work prob now.
yet I still want new ID in new IPTC  
([new_path[-1][i]] = costum4) ish  
  
and I want to hinder the vary long strings sneaking into a few IPTC cats.  
(if oldinfo[i] and len(str(oldinfo[i])) < 10000:)

In [13]:
def get_IPTC_data(old_path, new_path, date, time): # must also take new_path... date can by int right now..
    
    """Put time, date and path info in (converted and moved) image IPTC. fits into handl_... loop functions
    Rember, no IPTC gets carried over by handl_ functions automatically"""
    
    # Load stuff ----------------------------------------------------------------------------------------
    
#     new_path = old_path.split('.')[0]+'TEST.jpg' # temp!! Must come from above
    
    # info from old file
    oldinfo = IPTCInfo(old_path, force=True)
        
    # now load new new path inof
    newinfo = IPTCInfo(new_path, force=True)
    
    
    # get old IPTCs -------------------------------------------------------------------------------------
    for i in IPTC_keys:
#         if not newinfo[i] and oldinfo[i]: # if newinfo is None or empty and old info is NOT empty or None
#         if oldinfo[i]: # and len(str(oldinfo[i])) < 1000: # new info should be empty any way. And relevant info comes here or after
        if oldinfo[i] and len(str(oldinfo[i])) < 1000: # new info should be empty any way. And relevant info comes here or after
            newinfo[i] = oldinfo[i] # if you sort the string list first, you could use enumarate and strings[j]
    
        else:
            pass
    
    # get meta and path info ----------------------------------------------------------------------------

    # get info from old path structure regarding photographer and pub-status
#     path_info = old_path.split("/")[4:6] # name and pub-status give the paths of JOHAN SPANNER ONLY
    path_info = old_path.split("/")#[4:7] # name and pub-status give the paths of JOHAN SPANNER ONLY + year

    
    # get meta 
#     meta_keys =  ['custom1','copyright notice','date created','time created']
#     strings = [str(path_info[1]), str(path_info[0]), str(date), str(time)] #(date and time) must be strings to get saved - or error. makes all strings to be safe.
    
    meta_keys =  ['custom1', 'custom2', 'custom3', 'custom4','date created','time created']
    strings = [str(path_info[4]), str(path_info[5]), str(path_info[6]), str(path_info[-1]), str(date), str(time)] #(date and time) must be strings to get saved - or error. makes all strings to be safe.
    # so now you get, name, status, year, original name, date, time
    
    for j,i in enumerate(meta_keys):
        if not newinfo[i]: # if newinfo is None or empty
            newinfo[i] = strings[j] # if you sort the string list first, you could use enumarate and strings[j]
    
    # Saving and cleaning --------------------------------------------------------------------------------
    
    # save with meta and path info
    newinfo.save()# save to same file
    
    #removes wierd residual file, which appears whan I save IPTCs
    os.remove(new_path + '~')

    #print test:
#     for i in IPTC_keys:
#         if newinfo[i]:
#             print(f'key: {i}, info: {newinfo[i]}\n')

In [14]:
# FITS WITH Get_IPTC_data - in some way!
# An then into the larger loop - which will mostly supply the old_path and new_path

def MoveConvertGetMeta(old_path, new_path): # will also need new_path in time!!
    
    "Uses the handl... functions to move and convert images while also getting meta on date and time"
        
    # needs an extentsion, so must com after verification
    ext = old_path.split('.')[-1].lower() # Last entry, in case som dot is in name. lowercase just to streamline..
    
    # at point you'll new_path, so handl_ext(old_path, new_path)
    if ext == 'tif':
        date, time = handl_tif(old_path, new_path)
#         print(date, time) # needs to do someting else then print: goes to IPTC
        
    elif ext == 'nef' or ext == 'cr2':
        date, time = handl_nef_n_cr2(old_path, new_path)
#         print(date, time) # needs to do someting else then print: goes to IPTC
    
    elif ext == 'jpg':
        date, time = handl_jpg(old_path, new_path)
#         print(date, time) # needs to do someting else then print: goes to IPTC
    
    else:
        pass #pass unknown format
    
    
    #Takes info from old IPTC (if any), meta data and path to moved and converted image IPTC
    try:
        get_IPTC_data(old_path, new_path, date, time)
    
    except:
        pass


In [15]:
def flatten_dirs(external_dir, flatten_dir):
    
    # could have some safty for is it crashes... Like start somewhere else the zero... But not now.
    
    time_start = time.time() # start time
    mk_flatten_dir(flatten_dir) # creates the new folder
    files_count = count_files(external_dir) # counts all files to be moved
    counter = 0 # counter for all iterations
    count_exceptions = 0 # count files not being move, due to them not being images.
    ID_front = "JS" # Johan Spanner
    one_loop_time_list = [1] # save the time of each rund to extimate full time and time left. 1 works as a slightly informative (but pretty unconsequntiel) prior
    
    print(f'\nstarts moving files from: {external_dir}\nto new shared parent dir: {flatten_dir}...\n')
    
    for root, dirs, files in os.walk(external_dir):
        
        for name in files:
            time_loop_start = time.time()
            
            total_time = round((np.average(one_loop_time_list) * files_count)/60,2)
            gone_time = np.sum(one_loop_time_list)/60
            left_time = round(total_time - gone_time,2)
            
            counter += 1
            print(f'Handling file {counter} of {files_count}. Estimated time total {total_time} minutes. Estimated time left {left_time} minutes \t\t', end= '\r')
                        
            old_path = os.path.join(root, name) #all the info for the IPTC should be in in old_path name
            
            try: # you only wnat to move actual images
                
                # verify that is an image
                img = Image.open(old_path)  # open the image file
                img.verify()  # verify that it is a good image, without decoding it.. quite fast
                img.close()
                                
                # new path
                count_move = counter - count_exceptions
                new_path = flatten_dir + "/" + ID_front + str(count_move) + ".jpg" # path and image ID - could also put the ID in the IPTC now you are at it.
                                
                # -----------------------------------------------------------------------
                # Moving day..
    
                with io.capture_output(stdout=False, stderr=True, display=False) as captured: # Don't print logs from IPTC reader
                    MoveConvertGetMeta(old_path, new_path)
                
                # ------------------------------------------------------------------------
                    
    

            except Exception as e: # Non images you just pass
        
                count_exceptions += 1
                pass
            
            time_loop_end = time.time()
            time_loop = time_loop_end - time_loop_start
            one_loop_time_list.append(time_loop)
                
    # Logging time it takes to complete operation..
    time_end = time.time()
    time_run = round((time_end - time_start)/60,2)
                    
    print(f'\n\nDone moving files to parent dir: {flatten_dir}...')

    print(f'File(s) moved: {counter-count_exceptions} out of {files_count} files')
    print(f'File(s) not moved (not images): {count_exceptions} out of {files_count} files')
    
    print(f'\nALL DONE!\nThe operation took {time_run} minutes')
    


# Running script

In [16]:
flatten_dirs(external_dir, flatten_dir)


starts moving files from: /mnt/f/OctoberJohanArchiveTest
to new shared parent dir: /mnt/f/OctoberJohanArchiveFlatten...

Handling file 512 of 512. Estimated time total 6.81 minutes. Estimated time left -0.0 minutes 		

Done moving files to parent dir: /mnt/f/OctoberJohanArchiveFlatten...
File(s) moved: 481 out of 512 files
File(s) not moved (not images): 31 out of 512 files

ALL DONE!
The operation took 6.8 minutes


# Testing AREA1:
DO NOT RUN ON ALL IMAGES - ONLY SUBSETS

Testing all images moved

In [ ]:
for root, dirs, files in os.walk(flatten_dir):
        
    for name in files:
        new_path = os.path.join(root, name)
        image_check(new_path)
        
        testinfo = IPTCInfo(new_path, force = True)
        
        for i in IPTC_keys:
            if testinfo[i]:
                print(f'key: {i}, info: {testinfo[i]}\n')


# Testing AREA2

Testing on a small subsample of images

In [ ]:
# submitted
test_imgSubTIF = "/mnt/f/OctoberJohanArchiveTestOld/Spanner/Submitted/DSC_0128.tif"
test_imgSubJPG = "/mnt/f/OctoberJohanArchiveTestOld/Spanner/Submitted/DSC_0906.jpg"
test_imgSubjpgIP = "/mnt/f/OctoberJohanArchiveTestOld/Spanner/Submitted/DSC_0906.jpg"


# raw
test_imgRawNEF = "/mnt/f/OctoberJohanArchiveTestOld/Spanner/Raw/DSC_0001.NEF"
test_imgRawCR2 = "/mnt/f/OctoberJohanArchiveTestOld/Spanner/Raw/IMG_0001.CR2"
test_imgRawJPG = "/mnt/f/OctoberJohanArchiveTestOld/Spanner/Raw/P3300001.JPG"

# list
old_paths = [test_imgSubTIF, test_imgRawNEF, test_imgRawCR2, test_imgRawJPG, test_imgSubjpgIP, 'falseflag.todd']

In [ ]:
with io.capture_output(stdout=False, stderr=True, display=False) as captured: # Don't print logs from IPTC reader
    for old_path in old_paths:
        new_path = old_path.split('.')[0]+'TEST.jpg' #men skal så over i flatten med ny ID!!
        print(old_path +'___'+new_path )
        MoveConvertGetMeta(old_path, new_path)
        try:
            image_check(new_path)
        except:
            pass

In [ ]:
# TEST!

for old_path in old_paths:
    try:
        new_path = old_path.split('.')[0]+'TEST.jpg'
        testinfo = IPTCInfo(new_path, force = True)
        print(new_path)
        
        for i in IPTC_keys:
            if testinfo[i]:
                print(f'key: {i}, info: {testinfo[i]}\n')
        
    except:
        print('less cooool')
        pass

# Note that you have no warnings that these are not JPG! Which is nice